# Applying BespokeFit to the mop-terphenyl system

This notebook will be used to generate OpenFF force field for the mop-terphenyl system. We use mBuild to construct a short 3-mer of the polymer and get it's charges using AM1-BCC charges using OpenEye. Then using the BespokeFit workflow we will assign torsions/angles. Bespoke fit uses QM calculations on fragments present in our molecule to assign new more accurate torsion potentials.

In [1]:
try:
    from openmm import app
except ImportError:
    from simtk.openmm import app

from openff.toolkit.topology import FrozenMolecule, Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import pdb
import os
import openbabel
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer
import rdkit

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy

In [2]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)(c3cccc(c3)[C@H](N)C))cc1', smiles = True, name="MOP")
cap_o = mb.load('CO', smiles = True)
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.212  -0.58   -0.0471]), 1 bonds, id: 140375971539168>
26 <H pos=([-0.0898 -0.3998  0.0731]), 1 bonds, id: 140375971539408>
27 <H pos=([-0.05   -0.163   0.1233]), 1 bonds, id: 140375971539648>
28 <H pos=([0.0677 0.2914 0.0327]), 1 bonds, id: 140375971539888>
29 <H pos=([-0.1634  0.6468  0.1882]), 1 bonds, id: 140375971540128>
30 <H pos=([-0.2546  0.5109  0.2639]), 1 bonds, id: 140375971540368>
31 <H pos=([-0.2903  0.5616  0.0926]), 1 bonds, id: 140375971540608>
32 <H pos=([-0.3495  0.3241  0.1313]), 1 bonds, id: 140375971540848>
33 <H pos=([-0.3838  0.0902  0.0722]), 1 bonds, id: 140375971540896>
34 <H pos=([-0.0312 -0.0175 -0.2343]), 1 bonds, id: 140375971561872>
35 <H pos=([ 0.1606 -0.1453 -0.3215]), 1 bonds, id: 140375971562112>
36 <H pos=([ 0.3677 -0.1653 -0.1902]), 1 bonds, id: 140375971562352>
37 <H pos=([0.1908 0.073  0.123 ]), 1 bonds, id: 140375971562592>
38 <H pos=([ 0.4982 -0.1177 -0.0019]), 1 bonds, id: 140375971562832>
39 <H pos=([ 0.3596 -0.2263  0.1593]), 1

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2723 -0.5048 -0.0963]), 3 bonds, id: 140375971652800> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3612 -0.5415 -0.1777]), 1 bonds, id: 140375971652896> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2496 -0.3619 -0.0677]), 3 bonds, id: 140375971653328> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[25, 40],
                  separation=.15,
                  replace=True,
                  # orientation = [[0,-1,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=3, sequence='A')
chain.energy_minimize(forcefield="GAFF")

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2772 -0.5063 -0.0953]), 3 bonds, id: 140375970960576> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3632 -0.5405 -0.1796]), 1 bonds, id: 140375743927872> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2468 -0.3591 -0.0708]), 3 bonds, id: 140375743927968> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in ReadMolecule
  This Mol2 file is non-standard. Problem with molecule: RES Cannot interpret atom types correctly, instead attempting to interpret atom type: O as elements instead.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in ReadMolecule
  This Mol2 file is non-standard. Problem with molecule: RES Cannot interpret atom types correctly, instead attempting to interpret atom type: O as elements instead.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in R

In [6]:
# Set residue labels for polymer
for label in chain.labels["monomer"]:
    label.name = "TRI"
for label in chain.labels["Compound"]:
    label.name = "CAP"


In [7]:
# Rename atoms using mbuild interface
counts = {}
for particle in chain.particles():
    atom_name = particle.name
    if not atom_name in counts.keys():
        counts[atom_name] = 1
    else:
        counts[atom_name] += 1
    particle.name = atom_name + str(counts[atom_name])

In [8]:
# Write to PDB
chain.save("mop_trimer_mbuild.pdb", overwrite = True, residues = ["TRI", "CAP"])

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C1 pos=([-0.2929 -0.4733 -0.0867]), 3 bonds, id: 140375970960576> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O1 pos=([-0.3886 -0.5207 -0.1475]), 1 bonds, id: 140375743927872> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C2 pos=([-0.2629 -0.3446 -0.0658]), 3 bonds, id: 140375743927968> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/si

# File conversion between `mbuild` and OpenFF

Ideally structures generated from `mbuild` could be directly imported into the OpenFF workflow, however they are not. We have to recover connectivity information and write structures out into `.mol` and `.pdb` files to put into the OpenFF forcefield assignment workflow.

In [9]:
# Load into RDKit to assign bond infomration in PDB file
rdmol = rdkit.Chem.rdmolfiles.MolFromPDBFile("mop_trimer_mbuild.pdb", removeHs=False)
for atom in rdmol.GetAtoms():
    ri = atom.GetPDBResidueInfo()
    ri.SetIsHeteroAtom(False)
rdkit.Chem.rdmolfiles.MolToPDBFile(rdmol, "mop_trimer_mbuild_bonds.pdb")

In [10]:
# OpenBabel to convert PDB to Mol format
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("pdb", "mol")
mol = openbabel.OBMol()
obConversion.ReadFile(mol, "mop_trimer_mbuild_bonds.pdb")
obConversion.WriteFile(mol, "mop_trimer_mbuild.mol")

/home/tfobe/Research/tutorials/bespoke-fit-workshop-2022/env-bespoke/lib/python3.9/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


True

# OpenFF Parameter Assignment

Now that we have our input `.mol` and `.pdb` files we can assign bonded and non-bonded parameters using OpenFF.

In [11]:
# Load in with OpenFF
tri_mop = Molecule.from_file("mop_trimer_mbuild.mol")

In [12]:
pdbfile = app.PDBFile("mop_trimer_mbuild_bonds.pdb")
omm_topology = pdbfile.topology
omm_topology

<Topology; 1 chains, 5 residues, 153 atoms, 161 bonds>

In [13]:
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[tri_mop]
)

In [31]:
# Modified OpenFF to increase maxAtoms for AM1BCC method to 500 atoms
if not os.path.exists('terphenyl_mop_trimer_charges.sdf'):
    tri_mop.assign_partial_charges(partial_charge_method="am1bcc")
    tri_mop.to_file('terphenyl_mop_trimer_charges.sdf', file_format = 'sdf')
else:
    tri_mop = Molecule.from_file('terphenyl_mop_trimer_charges.sdf')

# Submit to BespokeFit Executor

The BespokeFit workflow needs an SDF file with charges to find fragments and assign a bespoke set of force-field parameters for chemical systems. Make sure you have an instance of the Bespoke fit executor running in another terminal by running:

```
BEFLOW_OPTIMIZER_KEEP_FILES=True openff-bespoke executor launch \
    --n-fragmenter-workers 1                                    \
    --n-optimizer-workers  1                                    \
    --n-qc-compute-workers 1                                    \
    --qc-compute-n-cores   1                                    \
    --qc-compute-max-mem   1.5                                  \
    --directory            bespoke-executor
```

These flags determine how BespokeFit distributes processes to perform the bespokefit. The `BEFLOW_OPTIMIZER_KEEP_FILES=True` allows us to look at output files from the fit.

In [32]:
!BEFLOW_OPTIMIZER_KEEP_FILES=True openff-bespoke executor submit \
     --file                 "terphenyl_mop_trimer_charges.sdf" \
     --workflow             "default"                            \
     --default-qc-spec      xtb gfn2xtb none


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

1. preparing the bespoke workflow                                               
                                                                                
⠋ loading the molecules
[✓] 1 molecules found
building fitting schemas ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
[✓] fitting schemas generated
                                                                                
2. submitting the workflow                                                      
                                                                                
submitting tasks ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
[✓] the following workflows were submitted
┏━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ SMILES            ┃ NAME             ┃ FILE                             ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩


In [33]:
!openff-bespoke executor list


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

The following optimizations were found:
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ ID ┃ SMILES                                                        ┃ STATUS  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ 1  │ C[C@H](c1cccc(c1)c2cc(ccc2c3ccc(cc3)C(=O)N[C@H](C)c4cccc(c4)c │ waiting │
│    │ 5cc(ccc5c6ccc(cc6)C(=O)OC)OC)OC)NC(=O)c7ccc(cc7)c8ccc(cc8c9cc │         │
│    │ cc(c9)[C@@H](C)NC(=O)OC(C)(C)C)OC                             │         │
└────┴───────────────────────────────────────────────────────────────┴─────────┘


In [34]:
!openff-bespoke executor watch --id 1


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

⠙ fragmenting the moleculele
[✓] fragmentation successful
⠦ generating bespoke QC datata
[✓] qc-generation successful
⠦ optimizing the parametersrs
[✓] optimization successful


In [36]:
!openff-bespoke executor retrieve --id 1 --output "mop_trimer_bespoke_fit.json" --force-field "openff-2.0.0_bespoke_mop_trimer.offxml"


──────────────────────────────── OpenFF Bespoke ────────────────────────────────

[✓] the bespoke fit is finished
                                                                                
outputs have been saved to mop_trimer_bespoke_fit.json                          
                                                                                
                                                                                
the bespoke force field has been saved to openff-2.0.0_bespoke_mop_trimer.offxml
                                                                                
